In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

### Paths to list of subjects and images

In [2]:
project_dir = "/home/nikhil/projects/brain_changes/"

data_dir = "/home/nikhil/projects/brain_changes/data/"
code_dir = "/home/nikhil/projects/brain_changes/brain-diff/"

metadata_dir = f"{code_dir}/metadata/adni/"
download_dir = f"{metadata_dir}/downloads/"
collections_dir = f"{download_dir}/collections/"
nii_paths_dir = f"{download_dir}/nii_paths/"

# Data download CSVs (from LONI collections)
# Vincent's download
adni12_long = f"{collections_dir}/ADNI123_logitudinal_collection_8_04_2021.csv"

# Nikhil's download
adni12_baseline = f"{collections_dir}/ADNI1-2_Baseline_MP-RAGE_5_08_2022.csv"
adni123_post_ohbm_1 = f"{collections_dir}/ADNI123_post_ohbm_1_7_15_2022.csv"
adni123_post_ohbm_1 = f"{collections_dir}/ADNI123_post_ohbm_2_7_15_2022.csv"

# Yasser's download
adni3_long = f"{collections_dir}/ADNI3_T1_10_20_2020.csv"

download_collections = {"adni12_long":adni12_long, "adni12_baseline":adni12_baseline, "adni3_long":adni3_long, 
                        "adni123_post_ohbm_1":adni123_post_ohbm_1}

# nifti paths
nii_path_files = {
    "adni123_collated_paths": f"{nii_paths_dir}adni123_collated_nii_paths.txt"
}

# ADNIMERGE
adnimerge = f"{metadata_dir}/ADNIMERGE.csv"



### ADNIMERGE

In [18]:
adnimerge_df = pd.read_csv(adnimerge)
print(f"ADNIMERGE len: {len(adnimerge_df)}")

visit_codes = adnimerge_df["VISCODE"].unique()
print(f"number of unique visit codes: {len(visit_codes)}")

ADNI1_subjects = adnimerge_df[adnimerge_df["ORIGPROT"]=="ADNI1"]["PTID"].unique()
ADNI2_subjects = adnimerge_df[adnimerge_df["ORIGPROT"]=="ADNI2"]["PTID"].unique()
ADNI3_subjects = adnimerge_df[adnimerge_df["ORIGPROT"]=="ADNI3"]["PTID"].unique()

print(f"number of ADNI1 subjects: {len(ADNI1_subjects)}, number of ADNI2 subjects: {len(ADNI2_subjects)}, number of ADNI3 subjects: {len(ADNI3_subjects)}")

# example subjects:
sample_sub_id = "002_S_0295"
adnimerge_df[adnimerge_df["PTID"]==sample_sub_id]

ADNIMERGE len: 16014
number of unique visit codes: 35
number of ADNI1 subjects: 819, number of ADNI2 subjects: 790, number of ADNI3 subjects: 674


,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,...,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,update_stamp
669,295,ADNI1,ADNI1,002_S_0295,2,bl,2006-05-09,CN,84.8,Male,...,34.73,NaN,NaN,NaN,NaN,0.00000,0.00000,0,0,2022-06-22 16:41:46.0
670,295,ADNI1,ADNI1,002_S_0295,2,m06,2006-11-02,CN,84.8,Male,...,34.73,NaN,NaN,NaN,NaN,0.48460,5.80328,6,6,2022-06-22 16:41:46.0
671,295,ADNI1,ADNI1,002_S_0295,2,m12,2007-05-25,CN,84.8,Male,...,34.73,NaN,NaN,NaN,NaN,1.04312,12.49180,12,12,2022-06-22 16:41:46.0
672,295,ADNI1,ADNI1,002_S_0295,2,m24,2008-08-04,CN,84.8,Male,...,34.73,NaN,NaN,NaN,NaN,2.23956,26.81970,24,24,2022-06-22 16:41:46.0
673,295,ADNI1,ADNI1,002_S_0295,2,m36,2009-05-07,CN,84.8,Male,...,34.73,NaN,NaN,NaN,NaN,2.99521,35.86890,36,36,2022-06-22 16:41:46.0
674,295,ADNI1,ADNI1,002_S_0295,2,m48,2010-05-13,CN,84.8,Male,...,34.73,NaN,NaN,NaN,NaN,4.01095,48.03280,48,48,2022-06-22 16:41:46.0
5828,295,ADNI1,ADNI1,002_S_0295,2,m18,2007-11-12,CN,84.8,Male,...,34.73,NaN,NaN,NaN,NaN,1.51129,18.09840,18,18,2022-06-22 16:41:46.0
5829,295,ADNI1,ADNI1,002_S_0295,2,m30,2008-11-06,CN,84.8,Male,...,34.73,NaN,NaN,NaN,NaN,2.49692,29.90160,30,30,2022-06-22 16:41:46.0
5830,295,ADNI1,ADNI1,002_S_0295,2,m42,2009-11-18,CN,84.8,Male,...,34.73,NaN,NaN,NaN,NaN,3.52909,42.26230,42,42,2022-06-22 16:41:46.0
5831,295,ADNI1,ADNI1,002_S_0295,2,m54,2010-11-02,CN,84.8,Male,...,34.73,NaN,NaN,NaN,NaN,4.48460,53.70490,54,54,2022-06-22 16:41:46.0


### Count number of subjects for given visits

In [4]:
cohorts = ["ADNI1","ADNI2","ADNI3"]
visit_1 = "bl"
visit_2 = "m24"

PTID_bl = adnimerge_df[adnimerge_df["ORIGPROT"].isin(cohorts) & adnimerge_df["VISCODE"].isin([visit_1])]["PTID"].unique()
PTID_m24 = adnimerge_df[adnimerge_df["ORIGPROT"].isin(cohorts) & adnimerge_df["VISCODE"].isin([visit_2])]["PTID"].unique()

PTID_bl_and_m24 = set(PTID_bl) & set(PTID_m24)

print(f"n_subs, BL: {len(PTID_bl)}, m24: {len(PTID_m24)}, BL and m24: {len(PTID_bl_and_m24)}")

n_subs, BL: 2283, m24: 1479, BL and m24: 1479


### Image and subject ID dataframe

In [5]:
collection_df = pd.DataFrame()
for k,v in download_collections.items():
    print(f"Reading download collection: {k}")
    _df = pd.read_csv(v)
    _df = _df[_df["Type"]=="Original"]
    n_subjects = len(_df["Subject"].unique())
    n_scans = len(_df["Image Data ID"].unique())

    print(f"Number of unique subjects: {n_subjects}, images: {n_scans}")
    collection_df = collection_df.append(_df)

collection_df = collection_df.drop_duplicates(subset=["Subject","Image Data ID"], keep="last")

n_subjects = len(collection_df["Subject"].unique())
n_scans = len(collection_df["Image Data ID"].unique())
visits = np.sort(collection_df["Visit"].unique())

print('\nEntire available collections:')
print(f"Number of unique subjects: {n_subjects}, images: {n_scans} \nvisits: {visits}")

collection_df.head()

Reading download collection: adni12_long
Number of unique subjects: 1057, images: 3573
Reading download collection: adni12_baseline
Number of unique subjects: 780, images: 1240
Reading download collection: adni3_long
Number of unique subjects: 165, images: 650
Reading download collection: adni123_post_ohbm_1
Number of unique subjects: 615, images: 2763

Entire available collections:
Number of unique subjects: 1669, images: 7208 
visits: [  1   2   3   4   5   6   8  10  13  14  15  17  22  24  25  26  28  30
  32  34  36  50 101 102 103 104 105]


,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I395985,130_S_4982,AD,F,59,28,MRI,MPRAGE,Original,10/25/2013,DCM,NaN
1,I133856,041_S_1002,CN,F,79,6,MRI,MPRAGE,Original,1/14/2009,DCM,8/03/2021
4,I54724,099_S_0470,AD,F,88,4,MRI,MPRAGE,Original,5/17/2007,DCM,8/03/2021
6,I115699,073_S_0565,AD,M,77,6,MRI,MPRAGE,Original,8/07/2008,DCM,8/03/2021
7,I508115,041_S_4143,EMCI,F,68,34,MRI,MPRAGE,Original,8/25/2015,DCM,NaN


### Count baseline and follow-up subjects from donwload collections
- These are based on csv generated from LONI download collection  
- According to Vincent, baseline visits: ADNI1=v2, ADNI2, ADNI3=v3
- Currently matching acq date from the nii image file name to the adnimerge dates to assign visit. 

In [6]:
# baseline_df = collection_df[collection_df["Visit"].isin([2,3])]
baseline_df = collection_df.copy()
baseline_subject_ids = baseline_df["Subject"].unique()
print(f"n_baseline_subjects: {len(baseline_subject_ids)}")
baseline_df.head()

n_baseline_subjects: 1669


,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I395985,130_S_4982,AD,F,59,28,MRI,MPRAGE,Original,10/25/2013,DCM,NaN
1,I133856,041_S_1002,CN,F,79,6,MRI,MPRAGE,Original,1/14/2009,DCM,8/03/2021
4,I54724,099_S_0470,AD,F,88,4,MRI,MPRAGE,Original,5/17/2007,DCM,8/03/2021
6,I115699,073_S_0565,AD,M,77,6,MRI,MPRAGE,Original,8/07/2008,DCM,8/03/2021
7,I508115,041_S_4143,EMCI,F,68,34,MRI,MPRAGE,Original,8/25/2015,DCM,NaN


In [7]:
followup_df = collection_df.drop_duplicates(subset=["Subject","Acq Date"], keep="last")
 
scan_count_df = pd.DataFrame(followup_df["Subject"].value_counts())
scan_count_df.columns = ["n_scans"]

subject_ids_follow_up = scan_count_df[scan_count_df["n_scans"]>1]
n_follow_up_scans = len(subject_ids_follow_up)
print(f"number of subject with follow-up scans: {n_follow_up_scans}")

followup_df = followup_df[followup_df["Subject"].isin(baseline_subject_ids)]

followup_subject_ids = followup_df["Subject"].unique()
print(f"n_followup_subjects (with baseline scan): {len(followup_subject_ids)}")

followup_df.head()

number of subject with follow-up scans: 1420
n_followup_subjects (with baseline scan): 1669


,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I395985,130_S_4982,AD,F,59,28,MRI,MPRAGE,Original,10/25/2013,DCM,NaN
1,I133856,041_S_1002,CN,F,79,6,MRI,MPRAGE,Original,1/14/2009,DCM,8/03/2021
4,I54724,099_S_0470,AD,F,88,4,MRI,MPRAGE,Original,5/17/2007,DCM,8/03/2021
6,I115699,073_S_0565,AD,M,77,6,MRI,MPRAGE,Original,8/07/2008,DCM,8/03/2021
7,I508115,041_S_4143,EMCI,F,68,34,MRI,MPRAGE,Original,8/25/2015,DCM,NaN


## Read all available nii paths

In [8]:
nii_paths = []
for k,v in nii_path_files.items():
    print(f"Reading nii path file: {k}")
    nii_paths += list(pd.read_csv(v,header=None)[0].values)

print(f"number of nii paths: {len(nii_paths)}")
nii_paths[:5]

Reading nii path file: adni123_collated_paths
number of nii paths: 10419


['./ADNI123_post_ohbm_3_5/ADNI/141_S_0915/MPRAGE_Repeat/2006-11-19_16_52_31.0/S22747/ADNI_141_S_0915_MR_MPRAGE_Repeat__br_raw_20061121113714882_138_S22747_I30232.nii',
 './ADNI123_post_ohbm_3_5/ADNI/021_S_0141/MP-RAGE_REPEAT/2007-02-22_15_10_22.0/S27047/ADNI_021_S_0141_MR_MP-RAGE_REPEAT_br_raw_20070223105841388_13_S27047_I41505.nii',
 './ADNI123_post_ohbm_3_5/ADNI/012_S_1165/MPRAGE_AUTOSHIM_ON/2008-02-07_14_39_57.0/S45785/ADNI_012_S_1165_MR_MPRAGE_AUTOSHIM_ON_br_raw_20080215174700095_1_S45785_I90641.nii',
 './ADNI123_post_ohbm_3_5/ADNI/029_S_1073/MP-RAGE_REPEAT/2008-05-02_13_32_56.0/S49628/ADNI_029_S_1073_MR_MP-RAGE_REPEAT_br_raw_20080505110107015_85_S49628_I104614.nii',
 './ADNI123_post_ohbm_3_5/ADNI/029_S_1073/MP-RAGE_REPEAT/2008-12-08_13_53_22.0/S60654/ADNI_029_S_1073_MR_MP-RAGE_REPEAT_br_raw_20081209105641679_10_S60654_I129688.nii']

In [9]:
def get_subject_info_from_path(p):
    """ Parses nii file name to grab ptid, iid, and acq_data
    """
    nii_file_name = p.rsplit("/",1)[1]
    ptid = nii_file_name.split("_")[1] + "_S_" + nii_file_name.split("_")[3]
    acq_date = p.rsplit("/",3)[1].split("_")[0]
    img_id = nii_file_name[::-1].split(".")[1].split("_")[0][::-1]

    return nii_file_name, ptid, acq_date, img_id

def get_closest_visit_code(subject_df, acq_date):
    """ Finds the closest acq date match with ADNIMERGE
    """
    date_diff_list = (pd.to_datetime(subject_df['EXAMDATE'].values, yearfirst=True, format="%Y/%m/%d") - 
                pd.to_datetime(acq_date, yearfirst=True, format="%Y/%m/%d"))

    closest_date_idx = np.argmin(np.abs(date_diff_list))
    diff_in_days = np.min(np.abs(date_diff_list)).days
    closest_date = subject_df['EXAMDATE'].values[closest_date_idx]

    visit_code = subject_df[subject_df["EXAMDATE"]==closest_date]['VISCODE'].values[0]

    return visit_code, diff_in_days


def filter_scans(nii_df, str_match, iid_criterion):
    """ Filters and selects single scan per timepoint based on nii file names and IIDs
    """
    n_ptids = len(nii_df["PTID"].unique())

    assert n_ptids == 1
    
    nii_df = nii_df[nii_df["nii_file_name"].str.contains(str_match)].sort_values(by=["PTID", "visit_code", "IID"])

    visits = nii_df["visit_code"].unique()

    # print(f"number of visits: {len(visits)}")

    filtered_nii_df = pd.DataFrame()
    n_scans = []
    for v, visit in enumerate(visits):
        df = nii_df[nii_df["visit_code"]==visit].copy()
        n_scans.append(len(df))

        if iid_criterion == "first":
            filtered_nii_df = filtered_nii_df.append(df.iloc[0])

        elif iid_criterion == "last":
            filtered_nii_df = filtered_nii_df.append(df.iloc[-1])


        else:
            print("Unknown iid selection criterion")


    filtered_nii_df["available_scans"] = n_scans
    return filtered_nii_df

In [10]:
nii_path_df = pd.DataFrame()
for p in nii_paths:
    nii_file_name, ptid, acq_date, img_id = get_subject_info_from_path(p)

    subject_df = adnimerge_df[(adnimerge_df["PTID"]==ptid)]

    try:    
        visit_code, diff_in_days = get_closest_visit_code(subject_df, acq_date)

        df = pd.DataFrame(columns=["PTID","IID","visit_code","acq_date","diff_in_days","nii_file_name","nii_path"])
        df.loc[0] = [ptid, img_id, visit_code, acq_date, diff_in_days, nii_file_name,p]
        nii_path_df = pd.concat([nii_path_df,df],axis=0)
        
    except:
        print(f"Error for path: {p}")
        continue

nii_path_df_bl_m24 = nii_path_df[nii_path_df["visit_code"].isin(["bl","m24"])]

print(f"\nnumber of nii paths for bl and m24: {len(nii_path_df_bl_m24)}")

nii_path_df_bl_m24.head()


number of nii paths for bl and m24: 3331


,PTID,IID,visit_code,acq_date,diff_in_days,nii_file_name,nii_path
0,141_S_0915,I30232,bl,2006-11-19,19,ADNI_141_S_0915_MR_MPRAGE_Repeat__br_raw_20061...,./ADNI123_post_ohbm_3_5/ADNI/141_S_0915/MPRAGE...
0,029_S_1073,I129688,m24,2008-12-08,3,ADNI_029_S_1073_MR_MP-RAGE_REPEAT_br_raw_20081...,./ADNI123_post_ohbm_3_5/ADNI/029_S_1073/MP-RAG...
0,131_S_0441,I14092,bl,2006-04-24,15,ADNI_131_S_0441_MR_MP-RAGE_REPEAT_br_raw_20060...,./ADNI123_post_ohbm_3_5/ADNI/131_S_0441/MP-RAG...
0,131_S_0441,I103963,m24,2008-04-24,0,ADNI_131_S_0441_MR_MP-RAGE_REPEAT_br_raw_20080...,./ADNI123_post_ohbm_3_5/ADNI/131_S_0441/MP-RAG...
0,136_S_0186,I95117,m24,2008-03-03,0,ADNI_136_S_0186_MR_MP-RAGE_REPEAT_br_raw_20080...,./ADNI123_post_ohbm_3_5/ADNI/136_S_0186/MP-RAG...


In [11]:
sub = "099_S_4480"
nii_path_df[nii_path_df["PTID"]==sub]

,PTID,IID,visit_code,acq_date,diff_in_days,nii_file_name,nii_path
0,099_S_4480,I361248,m12,2013-02-26,0,ADNI_099_S_4480_MR_MPRAGE_br_raw_2013022608103...,./ADNI123_post_ohbm_2_1/ADNI/099_S_4480/MPRAGE...
0,099_S_4480,I330216,m06,2012-09-05,0,ADNI_099_S_4480_MR_MPRAGE_br_raw_2012090508072...,./ADNI123_post_ohbm_2_5/ADNI/099_S_4480/MPRAGE...
0,099_S_4480,I306249,m03,2012-05-24,0,ADNI_099_S_4480_MR_MPRAGE_br_raw_2012052407273...,./ADNI123_post_ohbm_2_4/ADNI/099_S_4480/MPRAGE...
0,099_S_4480,I306246,m03,2012-05-24,0,ADNI_099_S_4480_MR_MPRAGE_GRAPPA2_br_raw_20120...,./ADNI123_post_ohbm_2_6/ADNI/099_S_4480/MPRAGE...
0,099_S_4480,I285326,bl,2012-02-17,17,ADNI_099_S_4480_MR_MPRAGE_GRAPPA2_br_raw_20120...,./ADNI123_post_ohbm_2_6/ADNI/099_S_4480/MPRAGE...
0,099_S_4480,I330215,m06,2012-09-05,0,ADNI_099_S_4480_MR_MPRAGE_GRAPPA2_br_raw_20120...,./ADNI123_post_ohbm_2_6/ADNI/099_S_4480/MPRAGE...
0,099_S_4480,I361244,m12,2013-02-26,0,ADNI_099_S_4480_MR_MPRAGE_GRAPPA2_br_raw_20130...,./ADNI123_post_ohbm_2_6/ADNI/099_S_4480/MPRAGE...


In [12]:
adni_nii_df = pd.merge(nii_path_df_bl_m24,adnimerge_df[["PTID"]],how="inner",on="PTID") #["ORIGPROT","AGE","DX_bl"]
adni_nii_df = adni_nii_df.drop_duplicates()
adni_nii_df = adni_nii_df[adni_nii_df["nii_file_name"].str.contains("RAGE")].sort_values(by=["PTID","visit_code"])

visit_count_df = adni_nii_df["PTID"].value_counts().reset_index().rename(columns={"index":"PTID","PTID":"count"})
visit_count_df[visit_count_df["count"]>1]

,PTID,count
0,133_S_1170,8
1,012_S_1009,8
2,012_S_0689,8
3,100_S_0190,8
4,136_S_0196,7
...,...,...
1152,073_S_2225,2
1153,067_S_0077,2
1154,100_S_5280,2
1155,116_S_4855,2


### Identify subjects with missing visit

In [13]:
str_match = "RAGE"
iid_criterion = "last"

filtered_nii_df = pd.DataFrame()

for ptid in adni_nii_df["PTID"].unique():
    subject_nii_df = adni_nii_df[adni_nii_df["PTID"]==ptid]
    filtered_subject_nii_df = filter_scans(subject_nii_df, str_match, iid_criterion)
    filtered_nii_df = filtered_nii_df.append(filtered_subject_nii_df)

print(f"filtered_df shape: {len(filtered_nii_df)}")

filtered_nii_df[filtered_nii_df["available_scans"]>1].head()

filtered_df shape: 2485


,IID,PTID,acq_date,diff_in_days,nii_file_name,nii_path,visit_code,available_scans
1434,I114210,002_S_0295,2008-07-23,12.0,ADNI_002_S_0295_MR_MP-RAGE__br_raw_20080723212...,./ADNI123_post_ohbm_2_4/ADNI/002_S_0295/MP-RAG...,m24,2
19533,I115006,002_S_0413,2008-07-31,0.0,ADNI_002_S_0413_MR_MPRAGE_br_raw_2008073122391...,./ADNI123_post_ohbm_1_3/ADNI/002_S_0413/MPRAGE...,m24,2
9229,I116582,002_S_0559,2008-08-15,4.0,ADNI_002_S_0559_MR_MPRAGE_br_raw_2008082122391...,./ADNI123_post_ohbm_1_3/ADNI/002_S_0559/MPRAGE...,m24,2
3052,I16391,002_S_0619,2006-06-01,19.0,ADNI_002_S_0619_MR_MP-RAGE_REPEAT_br_raw_20060...,./ADNI123_post_ohbm_3_4/ADNI/002_S_0619/MP-RAG...,bl,2
3047,I116120,002_S_0619,2008-08-13,6.0,ADNI_002_S_0619_MR_MP-RAGE_REPEAT_br_raw_20080...,./ADNI123_post_ohbm_3_1/ADNI/002_S_0619/MP-RAG...,m24,2


In [14]:
visit_count_df = pd.DataFrame(filtered_nii_df["PTID"].value_counts()).reset_index().rename(columns={"index":"PTID","PTID":"count"})
single_visit_subjects = visit_count_df[visit_count_df["count"]==1]["PTID"]
multiple_visit_subjects = visit_count_df[visit_count_df["count"] == 2]["PTID"]
multiple_visit_df = filtered_nii_df[filtered_nii_df["PTID"].isin(multiple_visit_subjects)]

print(f"number of single visit subjects: {len(single_visit_subjects)}, multiple visit subjects: {len(multiple_visit_subjects)}")

number of single visit subjects: 281, multiple visit subjects: 1102


### Generate a list of subjects with missing visits that need to be downloaded again

In [15]:
# number of single visit subjects: 370, multiple visit subjects: 949
# number of single visit subjects: 245, multiple visit subjects: 1074
# number of single visit subjects: 281, multiple visit subjects: 1102

# missing_ptids = list(single_visit_subjects.values)
# with open(f'{metadata_dir}/adni_download_ptids.txt', 'w') as f:
#     for line in missing_ptids:
#         f.write(f"{line},")

### Save list of subjects with specific set of visits

In [17]:
# subject_list = "adni123_bl_m24_subject_ids.txt"
# with open(f'{metadata_dir}/subject_ids/{subject_list}', 'w') as f:
#     for line in multiple_visit_subjects:
#         f.write(f"{line}\n")

# multiple_visit_nii_paths = list(multiple_visit_df["nii_path"].values)
# nii_path_list = "adni123_bl_m24_nii_paths.txt"
# with open(f'{metadata_dir}/downloads/nii_paths/{nii_path_list}', 'w') as f:
#     for line in multiple_visit_nii_paths:
#         f.write(f"{line}\n")

### Generate participants.tsv
- BIDS Dir: /home/nikhil/projects/brain_changes/data/adni/bids

In [40]:
save_csv = False 

participant_df = adnimerge_df[(adnimerge_df["PTID"].isin(multiple_visit_subjects)) & (adnimerge_df["VISCODE"]=="bl")].copy()
participant_cols_dict = {"PTID":"participant_id",
                        "AGE":"age",
                        "PTGENDER":"sex",
                        "DX_bl":"group",
                        "COLPROT":"COLPROT",
                        "ORIGPROT":"ORIGPROT"}

participant_df = participant_df[participant_cols_dict.keys()].rename(columns=participant_cols_dict)

if save_csv:
    # repo metadata
    participant_df.to_csv(f'{metadata_dir}/participants.csv',index=None)
    # BIDS dir
    participant_df.to_csv(f'{data_dir}/adni/bids/participants.tsv', index=None, sep="\t") 

participant_df.head()

,participant_id,age,sex,group,COLPROT,ORIGPROT
1,011_S_0003,81.3,Male,AD,ADNI1,ADNI1
10,011_S_0005,73.7,Male,CN,ADNI1,ADNI1
15,100_S_0006,80.4,Female,LMCI,ADNI1,ADNI1
20,022_S_0007,75.4,Male,AD,ADNI1,ADNI1
22,011_S_0010,73.9,Female,AD,ADNI1,ADNI1


In [37]:
adnimerge_df

,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER
0,ADNI1,ADNI1,011_S_0002,11,bl,2005-09-08,CN,74.3,Male
1,ADNI1,ADNI1,011_S_0003,11,bl,2005-09-12,AD,81.3,Male
2,ADNI1,ADNI1,011_S_0003,11,m06,2006-03-13,AD,81.3,Male
3,ADNI1,ADNI1,011_S_0003,11,m12,2006-09-12,AD,81.3,Male
4,ADNI1,ADNI1,011_S_0003,11,m24,2007-09-12,AD,81.3,Male
...,...,...,...,...,...,...,...,...,...
16009,ADNI3,ADNI3,305_S_6871,305,m24,2022-07-13,EMCI,68.1,Female
16010,ADNI3,ADNI3,035_S_6927,35,m12,2022-07-07,AD,59.5,Female
16011,ADNI3,ADNI3,073_S_6952,73,bl,2021-07-26,SMC,67.3,Female
16012,ADNI3,ADNI3,014_S_7080,14,bl,2022-07-12,NaN,57.2,Female
